In [116]:
import casadi as ca

In [135]:
def derive_model(dt):
    # p, parameters
    thr_max = ca.SX.sym("thr_max")
    m = ca.SX.sym("m")
    cl = ca.SX.sym("cl")
    S = ca.SX.sym("S")
    rho = ca.SX.sym("rho")
    g = ca.SX.sym("g")

    p= ca.vertcat(
        thr_max,
        m,
        cl,
        S,
        rho,
        g
    )

    # states
    # x, state
    posx = ca.SX.sym("posx")
    velx = ca.SX.sym("velx")
    x = ca.vertcat(posx,velx)

    # input
    throttle_cmd = ca.SX.sym("throttle_cmd")
    u = ca.vertcat(throttle_cmd)


    # force and moment
    fx_b = thr_max*u[0]-velx
    ax_b = fx_b/m
    velx = (ax_b)*dt

    # states derivative
    posx_dot = velx
    velx_dot = ax_b
    xdot = ca.vertcat(posx_dot,velx_dot)


    # algebraic (these algebraic expressions are used during the simulation)
    z = ca.vertcat()
    alg = z


    f = ca.Function("f", [x, u, p], [xdot], ["x", "u", "p"], ["xdot"])

    dae = {"x": x, "ode": f(x, u, p), "p": p, "u": u, "z": z, "alg": alg}

    return locals()

In [136]:
pnew = [1, 0.1, 1.8, 1, 1.225, 9.8] # [m cl S] 
unew = [1] #throttle

model = derive_model(0.1)


In [137]:
F = ca.integrator('F', 'idas', model["dae"])
res = F(x0=[4,1], z0=0, u=[0], p=pnew)
res["xf"]

DM([3.9, 4.54042e-05])

In [138]:
F = ca.integrator('F', 'idas', model["dae"], 0,1)
res = F(x0=[2,1], z0=0, u=[0.5], p=pnew)
res["xf"]

DM([1.95, 0.500023])

In [123]:
type(res["xf"])

casadi.casadi.DM

In [142]:
import numpy as np
np.array(res["xf"]).reshape(-1)

array([1.95000226, 0.50002259])